In [ ]:
import pandas as pd
df = pd.read_csv("diabetes.csv") #https://www.kaggle.com/datasets/akshaydattatraykhare/diabetes-dataset

In [ ]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import torch.nn as nn

In [ ]:
class HDDataset(Dataset):

    def __init__(self):
        xy = np.loadtxt('diabetes.csv', delimiter=',', dtype=np.float32, skiprows=1)

        self.n_samples = xy.shape[0]

        self.x_data = torch.from_numpy(xy[:, 0:-1])
        self.y_data = torch.from_numpy(xy[:, 8])

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

In [ ]:
dataset = HDDataset()
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [ ]:
class LNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LNeuralNet, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear3 = nn.Linear(hidden_size, 10)
        self.linear4 = nn.Linear(10, 5)

        self.linear2 = nn.Linear(5, 1)

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear3(out)
        out = self.relu(out)
        out = self.linear4(out)
        out = self.relu(out)
        out = self.linear2(out)
        y_pred = torch.sigmoid(out)
        return y_pred

In [ ]:
def train(model, dataloader, loss_fn, optimizer):
    num_epochs = 50
    n_total_steps = len(dataloader)

    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(dataloader):
            # Forward pass
            y_pred = model(images)
            # print(y_pred)
            # print(labels)
            loss = loss_fn(y_pred, labels.unsqueeze(1))

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 10 == 0:
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

In [ ]:
def test(model, dataloader):
    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        for images, labels in dataloader:
            labels = labels
            outputs = model(images)
            predicted = torch.round(outputs.data)
            # print(labels)
            # print(outputs)
            # print(predicted)
            n_samples += labels.size(0)
            for i in range(0, labels.size(0)):
                if (labels[i].item() == predicted[i, :].item()):
                    n_correct += 1

        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy of the network on the test data: {acc} %')

In [ ]:
model = LNeuralNet(8, 25)
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
train(model, train_dataloader, loss_fn, optimizer)

Epoch [1/50], Step [10/10], Loss: 0.4931
Epoch [2/50], Step [10/10], Loss: 0.5527
Epoch [3/50], Step [10/10], Loss: 0.5843
Epoch [4/50], Step [10/10], Loss: 0.6515
Epoch [5/50], Step [10/10], Loss: 0.5519
Epoch [6/50], Step [10/10], Loss: 0.5886
Epoch [7/50], Step [10/10], Loss: 0.5125
Epoch [8/50], Step [10/10], Loss: 0.6241
Epoch [9/50], Step [10/10], Loss: 0.5578
Epoch [10/50], Step [10/10], Loss: 0.5026
Epoch [11/50], Step [10/10], Loss: 0.6798
Epoch [12/50], Step [10/10], Loss: 0.5110
Epoch [13/50], Step [10/10], Loss: 0.6735
Epoch [14/50], Step [10/10], Loss: 0.7097
Epoch [15/50], Step [10/10], Loss: 0.4953
Epoch [16/50], Step [10/10], Loss: 0.6028
Epoch [17/50], Step [10/10], Loss: 0.5476
Epoch [18/50], Step [10/10], Loss: 0.5646
Epoch [19/50], Step [10/10], Loss: 0.6343
Epoch [20/50], Step [10/10], Loss: 0.6419
Epoch [21/50], Step [10/10], Loss: 0.6972
Epoch [22/50], Step [10/10], Loss: 0.5334
Epoch [23/50], Step [10/10], Loss: 0.6010
Epoch [24/50], Step [10/10], Loss: 0.5207
E

In [ ]:
test(model, test_dataloader)

Accuracy of the network on the test data: 79.87012987012987 %


In [ ]:
def save(model):
    torch.save(model.state_dict(), 'model.pth')

save(model)
loaded_model = LNeuralNet(8, 25)
loaded_model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [ ]:
test(loaded_model, test_dataloader)

Accuracy of the network on the test data: 79.87012987012987 %
